In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.page_load_strategy = 'normal'
driver = webdriver.Chrome(options=options)
driver.get('http://www.google.com')
driver.quit()

### request 테스트(한 페이지만 로드해서 정보 가져오기에 좋음_selenium보다 빠름)
- 페이지 내에서 날짜, 좌석등급 정보를 바꿔줘야 하기 때문에 이번 상황에는 부적절

In [ ]:
import requests

url = 'https://www.google.com/travel/flights/search?tfs=CBwQAhoqEgoyMDIzLTA5LTAxag4IAxIKL20vMDMwcWIzdHIMCAMSCC9tLzBoc3FmQAFIAXABggELCP___________wGYAQI&tfu=EgYIARABGAA'
response = requests.get(url)
response.status_code

200

### selenium(페이지 내에서 선택, 클릭 등이 필요할 경우 사용하기 좋음)

In [7]:
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By

options = Options()
# options.page_load_strategy = 'normal'
driver = webdriver.Chrome(options=options)
driver.get('https://www.google.com/travel/flights/search?tfs=CBwQAhoqEgoyMDIzLTA5LTAxag4IAxIKL20vMDMwcWIzdHIMCAMSCC9tLzBoc3FmQAFIAXABggELCP___________wGYAQI&tfu=EgYIARABGAA')

time.sleep(1)

tckt = []
data = []
week = ['월', '화', '수', '목', '금', '토', '일']
d, j = 1, 4

for _ in range(30):
    for i in range(1,5):

        # 좌석등급 바꾸기
        gbox_path = '//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/c-wiz/div[1]/c-wiz/div[2]/div[1]/div/div[1]/div[3]/div/div/div'
        gbox = driver.find_element(By.XPATH, gbox_path)
        gbox.click()
        g_path = f'//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/c-wiz/div[1]/c-wiz/div[2]/div[1]/div/div[1]/div[3]/div/div/div/div[2]/ul/li[{i}]'
        g = driver.find_element(By.XPATH, g_path)
        g.click()

        time.sleep(1)

        # 현재 페이지 파싱
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 항공권 정보 긁어오기
        ticket_box = soup.select('ul.Rk10dc')
        tickets = ticket_box[0].select('li>div>div.yR1fYc>div>div:nth-of-type(2)')
        tickets.extend(ticket_box[1].select('li>div>div.yR1fYc>div>div:nth-of-type(2)'))

        # 좌석 등급
        grade = soup.select_one('span#i19').text
        # 날짜
        day = f'9월 {d}일 ({week[j]})'

        tckt = [day, grade]

        for one in tickets:
            for info in one.select('span, div'):
                if info.text == '':
                    pass
                elif info.text in tckt:
                    pass
                else:
                    tckt.append(info.text)
            data.append(tckt)
            tckt = [day, grade]

        time.sleep(1)


    # 날짜 바꾸기
    nd_path = '//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/c-wiz/div[1]/c-wiz/div[2]/div[1]/div/div[2]/div[2]/div/div/div[1]/div/div/div[1]/div/div[1]/div/div[3]'
    next_day = driver.find_element(By.XPATH, nd_path)
    next_day.click()
    
    d += 1
    j += 1

    if j > 6: j = 0

    time.sleep(1)

driver.quit()

### 좌석 등급 정보 가져오기, 날짜 정보 가져오기
- 날짜는 html 내에서 업데이트가 되지 않고 첫 페이지의 날짜로만 표시됨

In [279]:
grade = soup.select_one('span#i19').text
grade
# 날짜가 html상에서 업데이트 되지 않아 수동으로 업데이트
day = soup.select_one('input.TP4Lpb.eoY5cb.j0Ppje').get('value')
day

'9월 1일 (금)'

- 수동으로 페이지가 바뀔 때마다 날짜가 입력되도록 함

In [353]:
week = ['월', '화', '수', '목', '금', '토', '일']
i, j = 1, 4
for _ in range(10):
    if j > 6: j = 0

    day = f'9월 {i}일 ({week[j]})'
    i += 1
    j += 1

    print(day)

9월 1일 (금)
9월 2일 (토)
9월 3일 (일)
9월 4일 (월)
9월 5일 (화)
9월 6일 (수)
9월 7일 (목)
9월 8일 (금)
9월 9일 (토)
9월 10일 (일)


### ticket정보 list 저장
- ticket정보를 리스트로 저장하는 코드를 테스트 해보는 코드

In [573]:
# ticket_box = soup.select('ul.Rk10dc')
# tickets = ticket_box[0].select('li>div>div.yR1fYc>div>div:nth-of-type(2)')
# tickets.extend(ticket_box[1].select('li>div>div.yR1fYc>div>div:nth-of-type(2)'))

# 티켓별 정보를 tckt 리스트로 불러와 data 리스트 안에 넣어줌
# data = []
# tckt = [day, grade]

# for one in tickets:
#     for info in one.select('span, div'):
#         if info.text not in tckt:
#             tckt.append(info.text)
#         elif info.text == '':
#             pass
#         else:
#             pass
#     data.append(tckt)
#     tckt = [day, grade]

### 데이터프레임으로 만들고 정제

- 여러 가지 시도를 위해 깊은 복사를 해준다.

In [8]:
import copy
data1 = copy.deepcopy(data)

- 정보를 추출하는 과정에서 제대로 추출되지 않는 경우를 찾아 해당 인덱스의 항목 내용을 확인해 봄.

In [28]:
data1[0][:30] # 경유 1회
data1[1][:30]  # 경유 1회, 소요 25시간
data1[2][:30]  # 직항
data1[3][:30]  # 경유 2회, [11] : '함께 예약된~'
data1[5213][:30]  # 경유 2회, [11] : 항공사
data1[12][:30]  # 경유 2회, [11] : '함께 예약된~'

['9월 1일 (금)',
 '이코노미',
 '오후 11:409월 1일 (금), 오후 11:40\xa0–\xa0오후 12:10+29월 3일 (일), 오후 12:10함께 예약된 다구간 항공권이 여행에는 여러 항공사의 항공권이 포함되어 있습니다. 이용하지 못한 연결편은 Kiwi.com에서 보증할 수 있습니다. 자세히 알아보기필리핀항공, 필리핀 에어아시아',
 '오후 11:409월 1일 (금), 오후 11:40\xa0–\xa0오후 12:10+29월 3일 (일), 오후 12:10',
 '오후 11:409월 1일 (금), 오후 11:40',
 '오후 11:40',
 '9월 1일 (금), 오후 11:40',
 '오후 12:10+29월 3일 (일), 오후 12:10',
 '오후 12:10+2',
 '+2',
 '9월 3일 (일), 오후 12:10',
 '함께 예약된 다구간 항공권이 여행에는 여러 항공사의 항공권이 포함되어 있습니다. 이용하지 못한 연결편은 Kiwi.com에서 보증할 수 있습니다. 자세히 알아보기필리핀항공, 필리핀 에어아시아',
 '함께 예약된 다구간 항공권이 여행에는 여러 항공사의 항공권이 포함되어 있습니다. 이용하지 못한 연결편은 Kiwi.com에서 보증할 수 있습니다. 자세히 알아보기',
 '함께 예약된 다구간 항공권',
 '이 여행에는 여러 항공사의 항공권이 포함되어 있습니다. 이용하지 못한 연결편은 Kiwi.com에서 보증할 수 있습니다. 자세히 알아보기',
 '필리핀항공, 필리핀 에어아시아',
 '20시간 30분LAXLos Angeles International Airport–ICN인천국제공항',
 '20시간 30분',
 'LAXLos Angeles International Airport–ICN인천국제공항',
 'LAXLos Angeles International Airport',
 'LAX',
 'Los Angeles International Airport',
 'ICN인천국제공항',
 'ICN',
 '인천국제공항',
 '경유 1회1시간 35분 M

- 각 정보 추출하는 코드를 테스트

In [ ]:
# 경유 횟수 추출('₩'로 바꿔주어 가격 추출 가능)
# for i in data1:
#     for j in i:
#         if j.startswith('경유') and len(j)==5:
#             print(j)
#         elif j.startswith('직항'):
#             print(j)

# 소요 시간 추출
# for i in data1:
#     for j in i:
#         if j[:2].isdigit() and len(j) < 9:
#             print(j)

# 출발.도착 공항 추출
# import re
# for i in data1:
#     for j in i:
#         if re.match("^[a-zA-Z]+.*", j) and len(j) == 3:
#             print(j)

# 항공사 추출(11번째 항목에 따라 항공사 정보를 담은 인덱스가 달라짐)
# for i in data1:
#     if i[11].startswith('함께'):
#         print(i[15])
#     else:
#         print(i[11])

# 날짜와 요일 정보를 따로 불러오기
for i in data1:
    # print(i[0][:-4])  # 날짜
    print(i[0][-2:-1])  # 요일

- 최종 실행 코드
    - 아래 코드를 실행하여 새로운 리스트에 다시 정리해준다.

In [10]:
import re

one_t = []
data_li = []

for i in data1:
    one_t.append(i[0][:-4])  # 날짜
    one_t.append(i[0][-2:-1])  # 요일
    one_t.append(i[1])  # 좌석등급
    one_t.append(i[5])  # 출발시간
    one_t.append(i[10])  #도착 시간
    for j in i:
        if j.startswith('경유') and len(j)==5:
            one_t.append(j)  # 경유 횟수
        elif j.startswith('직항'):
            one_t.append(j)  # 직항
        elif j.startswith('₩'):
            one_t.append(j)  # 가격
        elif j[:2].isdigit() and (j.endswith('분') or j.endswith('시간')):
            one_t.append(j)  # 소요 시간

    # 항공사
    if i[11].startswith('함께'):
        one_t.append(i[15])
    else:
        one_t.append(i[11])

    for j in i:
        if re.match("^[a-zA-Z]+.*", j) and len(j) == 3:
            one_t.append(j)  # 출발.도착 공항
            
    data_li.append(one_t)
    one_t = []

data_li

[['9월 1일',
  '금',
  '이코노미',
  '오후 5:00',
  '9월 3일 (일), 오전 11:10',
  '26시간 10분',
  '경유 1회',
  '₩869,874',
  '중화항공',
  'LAX',
  'ICN',
  'TPE'],
 ['9월 1일',
  '금',
  '이코노미',
  '오후 12:50',
  '9월 3일 (일), 오전 5:50',
  '25시간',
  '경유 1회',
  '₩919,744',
  '필리핀항공',
  'LAX',
  'ICN',
  'MNL'],
 ['9월 1일',
  '금',
  '이코노미',
  '오후 12:40',
  '9월 2일 (토), 오후 5:35',
  '12시간 55분',
  '직항',
  '₩1,203,542',
  '아시아나',
  'LAX',
  'ICN'],
 ['9월 1일',
  '금',
  '이코노미',
  '오후 1:24',
  '9월 3일 (일), 오후 7:00',
  '37시간 36분',
  '경유 2회',
  '₩853,948',
  '스피릿항공, 하와이안항공',
  'LAX',
  'ICN',
  'LAS',
  'HNL'],
 ['9월 1일',
  '금',
  '이코노미',
  '오후 7:10',
  '9월 3일 (일), 오후 7:00',
  '31시간 50분',
  '경유 2회',
  '₩859,676',
  '스피릿항공, 하와이안항공',
  'LAX',
  'ICN',
  'LAS',
  'HNL'],
 ['9월 1일',
  '금',
  '이코노미',
  '오후 4:08',
  '9월 3일 (일), 오후 7:00',
  '34시간 52분',
  '경유 2회',
  '₩861,102',
  '스피릿항공, 하와이안항공',
  'LAX',
  'ICN',
  'LAS',
  'HNL'],
 ['9월 1일',
  '금',
  '이코노미',
  '오후 5:00',
  '9월 3일 (일), 오후 12:45',
  '27시간 45분',
  '경유 1회',
  '₩869,874',

- 정리한 리스트를 데이터 프레임 형태로 저장 후 csv 파일로 내보냄

In [13]:
import pandas as pd
columns = ['date','day','class','departure','arrival','time','landing','cost','airline','port_d','port_a','via1','via2','via3','via4']
df = pd.DataFrame(data_li, columns=columns)
df
# df.to_csv("LAtoKR.csv")

,date,day,class,departure,arrival,time,landing,cost,airline,port_d,port_a,via1,via2,via3,via4
0,9월 1일,금,이코노미,오후 5:00,"9월 3일 (일), 오전 11:10",26시간 10분,경유 1회,"₩869,874",중화항공,LAX,ICN,TPE,None,None,None
1,9월 1일,금,이코노미,오후 12:50,"9월 3일 (일), 오전 5:50",25시간,경유 1회,"₩919,744",필리핀항공,LAX,ICN,MNL,None,None,None
2,9월 1일,금,이코노미,오후 12:40,"9월 2일 (토), 오후 5:35",12시간 55분,직항,"₩1,203,542",아시아나,LAX,ICN,None,None,None,None
3,9월 1일,금,이코노미,오후 1:24,"9월 3일 (일), 오후 7:00",37시간 36분,경유 2회,"₩853,948","스피릿항공, 하와이안항공",LAX,ICN,LAS,HNL,None,None
4,9월 1일,금,이코노미,오후 7:10,"9월 3일 (일), 오후 7:00",31시간 50분,경유 2회,"₩859,676","스피릿항공, 하와이안항공",LAX,ICN,LAS,HNL,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31051,9월 30일,토,퍼스트,오후 7:22,"10월 2일 (월), 오후 10:25",35시간 3분,경유 2회,"₩23,646,339","유나이티드항공, 일본항공, 아시아나아메리칸항공, 전일본공수운항사: Skywest D...",BUR,GMP,SFO,HND,None,None
31052,9월 30일,토,퍼스트,오전 12:30,"10월 1일 (일), 오후 5:40",25시간 10분,경유 1회,"₩26,661,049","캐세이패시픽항공, 아시아나",LAX,ICN,HKG,None,None,None
31053,9월 30일,토,퍼스트,오후 11:30,"10월 2일 (월), 오후 5:40",26시간 10분,경유 1회,"₩26,661,049","캐세이패시픽항공, 아시아나",LAX,ICN,HKG,None,None,None
31054,9월 30일,토,퍼스트,오전 12:30,"10월 2일 (월), 오전 7:10",38시간 40분,경유 2회,"₩28,206,989","캐세이패시픽항공, 말레이항공",LAX,ICN,HKG,KUL,None,None


- 저장된 csv파일 확인

In [16]:
pd.read_csv('LAtoKR.csv', index_col=0)

,date,day,class,departure,arrival,time,landing,cost,airline,port_d,port_a,via1,via2,via3,via4
0,9월 1일,금,이코노미,오후 5:00,"9월 3일 (일), 오전 11:10",26시간 10분,경유 1회,"₩869,874",중화항공,LAX,ICN,TPE,NaN,NaN,NaN
1,9월 1일,금,이코노미,오후 12:50,"9월 3일 (일), 오전 5:50",25시간,경유 1회,"₩919,744",필리핀항공,LAX,ICN,MNL,NaN,NaN,NaN
2,9월 1일,금,이코노미,오후 12:40,"9월 2일 (토), 오후 5:35",12시간 55분,직항,"₩1,203,542",아시아나,LAX,ICN,NaN,NaN,NaN,NaN
3,9월 1일,금,이코노미,오후 1:24,"9월 3일 (일), 오후 7:00",37시간 36분,경유 2회,"₩853,948","스피릿항공, 하와이안항공",LAX,ICN,LAS,HNL,NaN,NaN
4,9월 1일,금,이코노미,오후 7:10,"9월 3일 (일), 오후 7:00",31시간 50분,경유 2회,"₩859,676","스피릿항공, 하와이안항공",LAX,ICN,LAS,HNL,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31051,9월 30일,토,퍼스트,오후 7:22,"10월 2일 (월), 오후 10:25",35시간 3분,경유 2회,"₩23,646,339","유나이티드항공, 일본항공, 아시아나아메리칸항공, 전일본공수운항사: Skywest D...",BUR,GMP,SFO,HND,NaN,NaN
31052,9월 30일,토,퍼스트,오전 12:30,"10월 1일 (일), 오후 5:40",25시간 10분,경유 1회,"₩26,661,049","캐세이패시픽항공, 아시아나",LAX,ICN,HKG,NaN,NaN,NaN
31053,9월 30일,토,퍼스트,오후 11:30,"10월 2일 (월), 오후 5:40",26시간 10분,경유 1회,"₩26,661,049","캐세이패시픽항공, 아시아나",LAX,ICN,HKG,NaN,NaN,NaN
31054,9월 30일,토,퍼스트,오전 12:30,"10월 2일 (월), 오전 7:10",38시간 40분,경유 2회,"₩28,206,989","캐세이패시픽항공, 말레이항공",LAX,ICN,HKG,KUL,NaN,NaN
